In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix
!pip install implicit

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906.0,1.0,1.39,364.0,-0.6,1631.0,1.0,0.0,0.0
1,2375,26984851472,1,1033142.0,1.0,0.82,364.0,0.0,1631.0,1.0,0.0,0.0


In [13]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [14]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867.0, 834484.0, 856942.0, 865456.0, 88924..."
1,3,"[835476.0, 851057.0, 872021.0, 878302.0, 87994..."


### 1.1 Random recommendation

In [15]:
def random_recommendation(items, n=5):
    """Случайные рекомендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [16]:
# %%time

items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id']\
                .map(lambda x: \
                random_recommendation(items, n=5))

result.head(2)

,user_id,actual,random_recommendation
0,1,"[821867.0, 834484.0, 856942.0, 865456.0, 88924...","[12352232.0, 2442733.0, 909338.0, 12525346.0, ..."
1,3,"[835476.0, 851057.0, 872021.0, 878302.0, 87994...","[15596765.0, 990944.0, 868006.0, 946653.0, 101..."


### 1.2 Popularity-based recommendation

In [17]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [18]:
# %%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].map(lambda x: popular_recs)
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867.0, 834484.0, 856942.0, 865456.0, 88924...","[12352232.0, 2442733.0, 909338.0, 12525346.0, ...","[6534178.0, 6533889.0, 1029743.0, 6534166.0, 1..."
1,3,"[835476.0, 851057.0, 872021.0, 878302.0, 87994...","[15596765.0, 990944.0, 868006.0, 946653.0, 101...","[6534178.0, 6533889.0, 1029743.0, 6534166.0, 1..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [19]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
        
    random_id_items = np.random.choice(items_weight.index, n)
    random_items = items_weight[random_id_items]
    sum_weights = items_weight[random_id_items].sum() 
    df = pd.DataFrame({'sum_value': random_items,
                          'weight': [i / sum_weights for i in random_items]},
                          index=random_items.index
                          )
    recs = df.sort_values('weight', ascending=False).index
   
    return recs.tolist()

In [20]:
%%time
items_value = data_train.groupby('item_id')['sales_value'].sum()
items_weight = np.log(items_value)
# your_code

result['weighted_random_recommendation'] = result['user_id'].map(lambda x: weighted_random_recommendation(items_weight))
result.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


CPU times: user 5.06 s, sys: 21.6 ms, total: 5.08 s
Wall time: 6.92 s


In [26]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867.0, 834484.0, 856942.0, 865456.0, 88924...","[12352232.0, 2442733.0, 909338.0, 12525346.0, ...","[6534178.0, 6533889.0, 1029743.0, 6534166.0, 1...","[1052913.0, 873207.0, 944866.0, 1119234.0, 687..."
1,3,"[835476.0, 851057.0, 872021.0, 878302.0, 87994...","[15596765.0, 990944.0, 868006.0, 946653.0, 101...","[6534178.0, 6533889.0, 1029743.0, 6534166.0, 1...","[1056746.0, 916590.0, 1083493.0, 985361.0, 102..."


### Задание 2. Расчет метрик
Рассчитайте Precision@5, Precision@3, Recall@5 для каждого алгоритма с помощью функций из вебинара 1. Какой алгоритм показывает лучшее качество?

In [23]:
# your_code
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    flags = np.isin(recommended_list,bought_list)
    
    recall = flags[:k].sum() / len(bought_list)
    
    return recall


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [25]:
out = {'precision': [], 'recall': []}
for column in range(2,5):
  prec = result.apply(lambda x: precision_at_k(x[column],x[1],5),1).mean()
  rec = result.apply(lambda x: recall_at_k(x[column],x[1],5),1).mean()
  out['precision'].append(prec)
  out['recall'].append(rec)

pd.DataFrame(out, index= ['random_recommendation', 'popular_recommendation', 'weighted_random_recommendation']).T

,random_recommendation,popular_recommendation,weighted_random_recommendation
precision,0.000392,0.155240,0.000686
recall,0.000025,0.024996,0.000048


**Вывод по метрикам:** взвешенный рандомные рекомендации лучше обычных рандомных, но хуже полулярных 


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 (или другое количество) товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [24]:
# your_code